In [31]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
import sqlite3

In [32]:
load_dotenv()
# Set OpenAI API key
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
Model = "gpt-4.1-mini"
openai = OpenAI()
DB = "prices.db"

In [33]:
system_message = """
You are a helpful assistant for an Airline called FlightAI.
Give short, courteous answers, no more than 1 sentence.
Always be accurate. If you don't know the answer, say so.
"""

In [34]:
def get_ticket_price(city):
    print(f"DATABASE TOOL CALLED: Getting price for {city}", flush=True)
    with sqlite3.connect(DB) as conn:
        cursor = conn.cursor()
        cursor.execute('SELECT price FROM prices WHERE city = ?', (city.lower(),))
        result = cursor.fetchone()
        return f"Ticket price to {city} is ${result[0]}" if result else "No price data available for this city"

get_ticket_price("London")  # Test the function

DATABASE TOOL CALLED: Getting price for London


'Ticket price to London is $799.0'

In [35]:
price_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}
tools = [{"type": "function", "function": price_function}]
tools

[{'type': 'function',
  'function': {'name': 'get_ticket_price',
   'description': 'Get the price of a return ticket to the destination city.',
   'parameters': {'type': 'object',
    'properties': {'destination_city': {'type': 'string',
      'description': 'The city that the customer wants to travel to'}},
    'required': ['destination_city'],
    'additionalProperties': False}}}]

In [36]:
def chat(message,history):
    history = [{"role":h["role"],"content":h["content"]} for h in history]
    messages = [{"role":"system", "content":system_message}] + history + [{"role":"user","content":message}]
    response = openai.chat.completions.create(
        model= Model,
        messages = messages,
        tools = tools
    )
    while response.choices[0].finish_reason=="tool_calls":
        # get current message
        message = response.choices[0].message
        print(message)
        responses = handle_tool_calls(message)
        messages.append(message)
        messages.extend(responses)
        response = openai.chat.completions.create(
            model= Model,
            messages = messages,
            tools = tools
        )
    return response.choices[0].message.content
    #.choices[0].message.content

In [37]:
def handle_tool_calls(message):
    responses = []
    print(f"tool calls:{message.tool_calls}")
    for tool_call in message.tool_calls:
        if tool_call.function.name == "get_ticket_price":
            arguments = json.loads(tool_call.function.arguments)
            city = arguments.get('destination_city')
            price_details = get_ticket_price(city)
            responses.append({
                "role": "tool",
                "content": price_details,
                "tool_call_id": tool_call.id
            })
    return responses

In [38]:
gr.ChatInterface(chat,type="messages").launch()

* Running on local URL:  http://127.0.0.1:7895
* To create a public link, set `share=True` in `launch()`.


ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_nedQQ256qqB0b9hXs2OHBh1D', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')])
tool calls:[ChatCompletionMessageFunctionToolCall(id='call_nedQQ256qqB0b9hXs2OHBh1D', function=Function(arguments='{"destination_city":"London"}', name='get_ticket_price'), type='function')]
DATABASE TOOL CALLED: Getting price for London
